In [1]:
! pip install librosa numpy pandas demucs soundfile pyfluidsynth
! sudo yum install libsndfile
import librosa
import numpy as np
import os
import pandas as pd
import io
from pathlib import Path
from shutil import rmtree
import subprocess as sp
import soundfile as sf

from dataclasses import dataclass
from mido import MidiFile

@dataclass
class DeMixedAudio4():
    sample_rate: int
    bass : np.ndarray
    drums : np.ndarray
    other : np.ndarray
    vocals : np.ndarray

@dataclass
class DeMixedAudio6():
    sample_rate: int
    bass : np.ndarray
    drums : np.ndarray
    other : np.ndarray
    vocals : np.ndarray
    piano : np.ndarray
    guitar : np.ndarray
import mido
@dataclass
class ProcessedSong4():
    stems:DeMixedAudio4
    vocal_based_midi: mido.midifiles.midifiles.MidiFile
    melody_based_midi:mido.midifiles.midifiles.MidiFile
    full_song_based_midi:mido.midifiles.midifiles.MidiFile
    
@dataclass
class ProcessedSong6():
    stems:DeMixedAudio6
    vocal_based_midi: mido.midifiles.midifiles.MidiFile
    melody_based_midi:mido.midifiles.midifiles.MidiFile
    full_song_based_midi:mido.midifiles.midifiles.MidiFile
    

Package libsndfile-1.0.25-12.amzn2.1.x86_64 already installed and latest version
Nothing to do


In [2]:
def get_stems(path_to_file,model='htdemucs',output_path='../resources/tmp',delete_temp_files=True):
    file_name = os.path.basename(path_to_file)
    sp.run(["python3", "-m", "demucs.separate", "-o", output_path, "-n", model, path_to_file])
    # print("Demucs done, loading files")
    stems={}
    if model in ['htdemucs','htdemucs_ft']:
        for stem in ['bass','drums','other','vocals']:
            x,sr = librosa.load(os.path.join(output_path, model,file_name[:file_name.find('.')],f"{stem}.wav"), sr=None)
            stems[stem] = x
        if delete_temp_files:
            print("Deleting temp files")
            rmtree(f"{output_path}/{model}/{file_name[:file_name.find('.')]}")
        return DeMixedAudio4(sr, bass=stems['bass'], drums=stems['drums'], other=stems['other'], vocals=stems['vocals'])#stems
    elif model == 'htdemucs_6s':
        for stem in ['bass','drums','other','vocals','piano', 'guitar']:
            x,sr = librosa.load(os.path.join(output_path, model,file_name[:file_name.find('.')],f"{stem}.wav"), sr=None)
            stems[stem] = x
        if delete_temp_files:
            print("Deleting temp files")
            rmtree(f"{output_path}/{model}/{file_name[:file_name.find('.')]}")
        return DeMixedAudio6(sr, bass=stems['bass'], drums=stems['drums'], other=stems['other'], vocals=stems['vocals'], piano=stems['piano'], guitar=stems['guitar'])#stems
    
def get_piano(path_to_file, composer='composer1', delete_temp_files=True):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    config = OmegaConf.load("pop2piano/config.yaml")
    wrapper = TransformerWrapper(config)
    wrapper = wrapper.load_from_checkpoint("model-1999-val_0.67311615.ckpt", config=config).to(device)
    model = "dpipqxiy"

    wrapper.eval()
    pm, composer, mix_path, midi_path = wrapper.generate(
        audio_path=path_to_file, 
        composer=composer, 
        model=model,
        show_plot=False, 
        save_midi=True, 
        save_mix=False, 
    )
    midi_file = MidiFile(midi_path)
    if delete_temp_files:
        os.remove(midi_path)
    
    return midi_file

def process_file(audio_file:np.ndarray, sample_rate:int, song_name:str, output_folder_path:str='../resources/tmp', demucs_model='htdemucs_6s', delete_temp_files=True):
    if not os.path.exists(output_folder_path):
        os.mkdir(output_folder_path)
    
    # Write array to audio file for demucs and pop2piano
    sf.write(os.path.join(output_folder_path,f'{song_name}.wav'), audio_file, sample_rate)
    # if not os.path.exists(os.path.join(output_folder_path, demucs_model, song_name)):
    # Separate stems using demucs
    print("Running demucs")
    song = get_stems(path_to_file=os.path.join(output_folder_path,f'{song_name}.wav'), model=demucs_model,output_path=output_folder_path, delete_temp_files=delete_temp_files)
    # Save the melodic line (guitar+bass+other+piano)
    if demucs_model == 'htdemucs_6s':
        melodic_line = song.guitar + song.bass+ song.other + song.piano
    if demucs_model in ['htdemucs','htdemucs_ft']:
        melodic_line = song.other+song.bass
    sf.write(os.path.join(output_folder_path,f'{song_name}_melodicline.wav'),melodic_line, song.sample_rate)
    vocal = song.vocals
    sf.write(os.path.join(output_folder_path,f'{song_name}_vocal.wav'),vocal, song.sample_rate)

    # Run pop2piano on full song, melodic line and vocal only
    print("Running pop2piano")
    full_song_midi = get_piano(os.path.join(output_folder_path,f'{song_name}.wav'),composer='composer1', delete_temp_files=delete_temp_files)
    melodicline_midi = get_piano(os.path.join(output_folder_path,f'{song_name}_melodicline.wav'),composer='composer1', delete_temp_files=delete_temp_files)
    vocal_only_midi = get_piano(os.path.join(output_folder_path,f'{song_name}_vocal.wav'),composer='composer1', delete_temp_files=delete_temp_files)
    
    if demucs_model == 'htdemucs_6s':
        return ProcessedSong6(song, vocal_only_midi, melodicline_midi, full_song_midi)
    if demucs_model in ['htdemucs','htdemucs_ft']:
        return ProcessedSong4(song, vocal_only_midi, melodicline_midi, full_song_midi)
    

In [6]:
# the line "from transformer_wrapper import TransformerWrapper" throws an error if this is not used
get_stems(os.path.join(os.getcwd(),'..',f'resources/TakeOnMe.wav'), model='htdemucs_6s', delete_temp_files=False)

!apt-get install -y fluidsynth
!sudo yum install fluidsynth
!git clone https://github.com/sweetcocoa/pop2piano/
!cd pop2piano
!pip install pretty-midi==0.2.9 omegaconf==2.1.1 youtube-dl==2021.12.17 transformers==4.16.1 pytorch-lightning==1.8.4 essentia==2.1b6.dev609 note-seq==0.0.3 pyFluidSynth==1.3.0

if not os.path.exists(os.path.join(os.getcwd(),'model-1999-val_0.67311615.ckpt')):
    sp.run(["wget", "https://github.com/sweetcocoa/pop2piano/releases/download/dpi_2k_epoch/model-1999-val_0.67311615.ckpt"])
# !wget https://github.com/sweetcocoa/pop2piano/releases/download/dpi_2k_epoch/model-1999-val_0.67311615.ckpt

import os 
import sys
sys.path.append("pop2piano")
import glob
import random

import torch
import torchaudio

import numpy as np
import pandas as pd
import IPython.display as ipd
import soundfile as sf
# from google.colab import files

from tqdm.auto import tqdm
from omegaconf import OmegaConf
import note_seq

from utils.dsp import get_stereo
from utils.demo import download_youtube
from transformer_wrapper import TransformerWrapper
from midi_tokenizer import MidiTokenizer, extrapolate_beat_times
from preprocess.beat_quantizer import extract_rhythm, interpolate_beat_times

ffprobe: error while loading shared libraries: libopenh264.so.5: cannot open shared object file: No such file or directory


Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /home/ec2-user/SageMaker/scalable-asset-generation/resources/tmp/htdemucs_6s
Separating track /home/ec2-user/SageMaker/scalable-asset-generation/src/../resources/TakeOnMe.wav


100%|████████████████████████████████████████████████████████████████████████| 245.7/245.7 [00:10<00:00, 23.46seconds/s]


/bin/sh: apt-get: command not found
Loaded plugins: dkms-build-requires, extras_suggestions, langpacks, priorities,
              : update-motd, versionlock
https://download.docker.com/linux/centos/2/x86_64/stable/repodata/repomd.xml: [Errno 14] HTTPS Error 404 - Not Found
Trying other mirror.
neuron                                                   | 2.9 kB     00:00     
61 packages excluded due to repository priority protections
No package fluidsynth available.
Error: Nothing to do
fatal: destination path 'pop2piano' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [ ]:
x_test,sr_test = librosa.load('../resources/TakeOnMe.wav', sr=None)
f = process_file(x_test, sr_test, song_name='TakeOnMe',output_folder_path='../resources/tmp/test',demucs_model='htdemucs_ft')
f

In [7]:
f

ProcessedSong4(stems=DeMixedAudio4(sample_rate=44100, bass=array([1.5258789e-05, 1.5258789e-05, 1.5258789e-05, ..., 3.0517578e-05,
       3.0517578e-05, 3.0517578e-05], dtype=float32), drums=array([3.0517578e-05, 3.0517578e-05, 3.0517578e-05, ..., 3.0517578e-05,
       3.0517578e-05, 3.0517578e-05], dtype=float32), other=array([4.5776367e-05, 7.6293945e-05, 6.1035156e-05, ..., 3.0517578e-05,
       3.0517578e-05, 3.0517578e-05], dtype=float32), vocals=array([0.0000000e+00, 0.0000000e+00, 1.5258789e-05, ..., 3.0517578e-05,
       3.0517578e-05, 3.0517578e-05], dtype=float32)), vocal_based_midi=MidiFile(type=1, ticks_per_beat=384, tracks=[
  MidiTrack([
    MetaMessage('set_tempo', tempo=500000, time=0),
    MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0),
    MetaMessage('end_of_track', time=1)]),
  MidiTrack([
    Message('program_change', channel=0, program=0, time=0),
    Message('note_on', channel=0, note=74, velo